In [38]:
from dataAccess import *

In [4]:
class_dict = {}

In [5]:
CLASSES = [
    "BLG 101E",
    "BLG 113E",
    "BLG 102E",
    "MAT 210E",
    "BLG 231E",
    "EHB 222E",
    "MAT 281E"
    ] #ENTER YOUR CLASSES HERE
DEPT = "BLGE" #ENTER YOUR DEPARTMENT CODE HERE

In [39]:
CLASSES = [""]
for CLASS in CLASSES:
    class_dict[CLASS] = get_class(CLASS,DEPT)



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Driver [/Users/firatkizilboga/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache


Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/div[3]/table/tbody/tr[1]"}
  (Session info: chrome=105.0.5195.102)
Stacktrace:
0   chromedriver                        0x0000000102d68788 chromedriver + 4515720
1   chromedriver                        0x0000000102cec9d3 chromedriver + 4008403
2   chromedriver                        0x000000010297f12a chromedriver + 413994
3   chromedriver                        0x00000001029b6873 chromedriver + 641139
4   chromedriver                        0x00000001029b6a61 chromedriver + 641633
5   chromedriver                        0x00000001029e8f64 chromedriver + 847716
6   chromedriver                        0x00000001029d37fd chromedriver + 759805
7   chromedriver                        0x00000001029e6bd9 chromedriver + 838617
8   chromedriver                        0x00000001029d3603 chromedriver + 759299
9   chromedriver                        0x00000001029a9990 chromedriver + 58817

In [16]:
CLASS_DATES = {}
for CLASS in CLASSES:
    DATES = class_dict[CLASS].groupby(["Gün","Saat"]).groups.keys()
    CLASS_DATES[CLASS] = list(DATES)

In [17]:
from itertools import product
perms = list(product(*list(CLASS_DATES.values())))

In [19]:
days_dict = {"Pazartesi":"Monday","Salı":"Tuesday","Çarşamba":"Wednesday","Perşembe":"Thursday","Cuma":"Friday"}
class Lesson():
    def __init__(self, name, days, hours):
        self.name = name
        self.days = list(map(lambda x: days_dict[x], days.split("\n")))
        self.hours = hours.split("\n")
        self.hours = list(map(lambda x: x.split("/") ,self.hours))
    def __str__(self):
        return self.name
class HourAlreadyFilledError(Exception):
    """hour already filled"""
    pass
class TimeTable():
    def __init__(self):
        DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday"]
        self.__times = {}
        for d in DAYS:
            for hour in range(8,18):
                self.__times[d] = ["FREE"] * 9
        self.total = 0
    def fill_hours(self,lesson):
        for day,hour in zip(lesson.days,lesson.hours):
            start_hour, end_hour = map(lambda x: int(x[:2]),hour)
            for i in range(start_hour,end_hour):
                if self.__times[day][i-8] != "FREE":
                    raise HourAlreadyFilledError
                else:
                    self.__times[day][i-8] = lesson
        self.__df= pd.DataFrame(self.__times,index=["08:30-09:29","09:30-10:29","10:30-11:29","11:30-12:29","12:30-13:29","13:30-14:29","14:30-15:29","15:30-16:29","16:30-17:29"])
        return "SUCCESS"
    def __call__(self):
        return self.__df
    def score(self):
        FREE = "FREE"
        total = 0
        any_day_free = False
        for day in self.__times:
            classes = self.__times[day]
            is_day_free = all(FREE == x for x in classes)
            if is_day_free == True:
                any_day_free = True
            if is_day_free == True:
                pass #total += 200
            if classes[0] != FREE:
                total -= 50
            if classes[-1] != FREE:
                total -= 20
            if all(x not in classes for x in ("MAT 104","FIZ 102","FIZ 102EL","INS 102","KIM 101")) and (is_day_free is not True):
                total += 20
                for i in classes:
                    if i != FREE:
                        total+=50
            for i in range(len(classes)-2):
                if classes[i] != FREE and classes[i] != classes[i+1]:
                    if classes[i+1] != FREE:
                        total -= 20
        if any_day_free == False:
            pass #total = -1000
        self.total = total
    def __lt__(self, other):
        return self.total<other.total

In [33]:
t = 0
tables = []
for p in perms:
    lessons = []
    Tt = TimeTable()
    try:
        for c,o in zip(CLASSES,p):
            lessons.append(Lesson(c,*o))
        for lesson in lessons:
            Tt.fill_hours(lesson)
        Tt.score()
    except HourAlreadyFilledError:
        continue
    tables.append(Tt)

In [25]:
tables[0]()

IndexError: list index out of range

In [15]:
tables[1]()